<a href="https://colab.research.google.com/github/JFrankVC/aprendizaje_profundo/blob/main/Examen_01_DL_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: *José Francisco Villanueva Cortés*
# Dossier: *742015*

In [4]:
# imports
import torch.nn as nn
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import torch
from torchvision.models import

In [ ]:
# Utilities
WHITE = '\033[97m'
YELLOW = '\033[93m'
BLUE = '\033[94m'
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

# Zip status
zip_decompressed = False

In [ ]:
# Connect drive to verify model storage
# Mount the Google Drive
# Allow access - check steps and methods in: https://neptune.ai/blog/google-colab-dealing-with-files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)  # if Colab root directory is shown, go to /content folder

Mounted at /content/drive


In [7]:
# Check current working directory corresponds to Google Drive
retval = os.getcwd()
print(f"Current working directory {retval}")

# Change the directory - Google Drive
path = "/content/drive/MyDrive/Colab_DL/Examen_I/"
os.chdir(path)

# Check current working directory corresponds to Google Drive
retval = os.getcwd()
print(f"Current working directory {retval}")

Current working directory /content/drive/MyDrive/Colab_DL/Examen_I
Current working directory /content/drive/MyDrive/Colab_DL/Examen_I


In [6]:
# Decompress zip file into Drive itself (only once)
if (zip_decompressed == True):
    zip_path = "/content/drive/MyDrive/Colab_DL/Examen_I/m3ex02-data.zip"
    path = "/content/drive/MyDrive/Colab_DL/Examen_I/m3ex02-data/"

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path)

KeyboardInterrupt: 

### Get/read/generate data

### Create model

In [ ]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)